# A.9 Performance Tips（性能贴士）

- 把python loop和conditional logic（条件逻辑）转换为数组操作和布尔数组操作
- 尽可能是用广播
- 使用数组视图（切片）来方式复制数据
- 使用ufuncs和ufunc方法

# 1 The Importance of Contiguous Memory（邻近内存的重要性）

如果访问相邻的内存，速度会快很多。在内存中创建数组的时候，存储顺序有Fortran（列为主）和C（行为主）两种。默认，numpy数组是按C邻近的方式创建的。可以通过flags属性来查看：

In [1]:
import numpy as np

In [2]:
arr_c = np.ones((1000, 1000), order='C')
arr_f = np.ones((1000, 1000), order='F')

In [3]:
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [4]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [5]:
arr_f.flags.f_contiguous

True

在这个例子里，如果要计算每一行的和，理论上arr_c是比arr_f要快的，因为行在内存里是邻近的。我们可以检查一下：

In [6]:
%timeit arr_c.sum(1)

1000 loops, best of 3: 576 µs per loop


In [7]:
%timeit arr_f.sum(1)

1000 loops, best of 3: 672 µs per loop


当我们在numpy之外想要提高性能的话，这是个不错的选择。如果我们想要数组按我们想要的顺序在内存上进行存储，可以使用copy，并传入C或F：

In [8]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

当在一个数组上构建视图的时候，记住其结果不一定保证是相邻的：

In [9]:
arr_c[:50].flags.contiguous

True

In [10]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False